<a href="https://colab.research.google.com/drive/1Q-BS1MJum3WHEIhmulXSyFoNdbDg2R8c?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning SP 2022/2023

- Prof. Cesare Alippi
- Giorgia Adorni ([`giorgia.adorni@usi.ch`](mailto:giorgia.adorni@usi.ch))<br>
- Fatima Ezzeddine ([`fatima.ezzeddine@usi.ch`](mailto:fatima.ezzeddine@usi.ch))<br>
- Alessandro Manenti ([`alessandro.manenti@usi.ch`](mailto:alessandro.manenti@usi.ch))

---

# Lab 03: Model Performance

The objectives of the lab are as follows:

- Evaluate the performance of a model
- Perform splits on the data
- Assess which model is the best along many models
- Gain familiarity with the concept of models hyper-parameter tuning

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Auxiliary code -------------------- #

# function to plot decision boundaries
def plot_decision_surface(model, x, y, transform=lambda x:x):
    
  from matplotlib.colors import ListedColormap
  # color_maps
  cm = plt.cm.RdBu
  cols = ['#FF0000', '#0000FF']
  cm_bright = ListedColormap(cols)  

  #init figure
  fig = plt.figure()

  # Create mesh
  h = .1  # step size in the mesh
  x_min, x_max = x[:, 0].min() - .5, x[:, 0].max() + .5
  y_min, y_max = x[:, 1].min() - .5, x[:, 1].max() + .5
  xx, yy = np.meshgrid(np.arange(x_min, x_max, h), 
                       np.arange(y_min, y_max, h))

  # plot train data
  cy = [cols[int(yi)] for yi in y]
  plt.scatter(x[:, 0], x[:, 1], c=cy, cmap=cm_bright,
              edgecolors='k')
  plt.xlim(xx.min(), xx.max())
  plt.ylim(yy.min(), yy.max())

  plt.xlabel(r'$x_1$')
  plt.ylabel(r'$x_2$');

  y_pred = model.predict(transform(np.c_[xx.ravel(), yy.ravel()]))

  y_pred = y_pred.reshape(xx.shape)
  plt.contourf(xx, yy, y_pred > 0.5, cmap=cm, alpha=.5)

## Use-case scenario
Consider the following use-case scenario: A company has approached us with a request to develop a machine learning model for one of their machines. We have been provided with a labeled dataset consisting of $(x_i, y_i)$ for $i=1, ..., N$. 

Our objective is to identify the best possible model, denoted as $f(x; \hat \theta)$, and provide an estimate of its performance, which is represented as $V(\hat \theta)$.

In [ ]:
# Prepare some data
N = 400

np.random.seed(200402)
# generating data points by samling from a random distribution and then changin the means of the distributions by adding constants to the values
Xa = np.random.randn(N//4, 2)
Xb = np.random.randn(N//4, 2) + np.array([ 8.,  1.])
Xc = np.random.randn(N//4, 2) + np.array([-4., -1.])
Xd = np.random.randn(N//4, 2) + np.array([ 4., -1.])

#stacking the features together
X = np.vstack([Xa, Xb, Xc, Xd])

plt.scatter(X[:, 0], X[:, 1]);

Creates two classes of equal size, one consisting of the first half of the elements in y (which are all zero) and the other consisting of the second half of the elements in y (which are all one).

In [ ]:
# Assigning labels to our dataset

# Creates a NumPy array y of size N with all elements initialized to zero.
y = np.zeros((N,))

# Sets the second half of the elements in y to one. 
y[N//2:] = 1

# plot the data
plt.scatter(X[:, 0], X[:, 1], c=y);

In [ ]:
X.shape, y.shape

## Train some models

Let's start from a logistic regression ([sklearn doc](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)).


We will try 2 types of models:
- Logistic regression as the linear model
- Feed Forward Neural Network as the non-linear model

In [ ]:
from sklearn.linear_model import LogisticRegression

#initiate the class
logreg = LogisticRegression()
#feed it with X and y
logreg.fit(X, y)
#plot the decision surface
plot_decision_surface(model=logreg, x=X, y=y)

Let's try a feed-forward neural net ([keras doc](https://keras.io/models/sequential/)). 

We will use 3 keras component to build our FFNN
- Sequential class
- Dense class
- Binary Cross Entropy 

Binary cross entropy is a common loss function used in machine learning, particularly for binary classification tasks where the goal is to predict one of two possible outcomes. It measures the difference between the predicted probabilities and the actual binary labels of a given dataset.

When training neural networks for binary classification, we take the loss to be the __cross-entropy error function__: 

$$
L({\boldsymbol \theta}) =  -\frac1n \sum_{i=1}^n \bigg[y_i  \log \hat y_i + (1 - y_i)  \log (1 - \hat y_i)\bigg]
$$

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.losses import BinaryCrossentropy

def create_ffnn(neurons=3, activation="tanh"):
    # sequential class
    model = Sequential()
    # Dense class
    model.add(Dense(neurons, input_shape=(2,), activation=activation))
    model.add(Dense(1, activation="sigmoid"))
    # binary cross entropy    
    model.compile(loss=BinaryCrossentropy(), metrics=["accuracy"])
    return model

epochs = 300

ffnn = create_ffnn()
ffnn.fit(X, y, epochs=epochs, verbose=0)

plot_decision_surface(model=ffnn, x=X, y=y)

## Performance assessment

In [ ]:
# Set sizes
n = int(N * .8)  # Training points, 80%
l = N - n        # Test points, the resting 20%
print("num training points: n=",  n)
print("num test points:     l= ", l)

# Data split
X_train, y_train = X[:n], y[:n]
X_test, y_test = X[n:], y[n:]

In [ ]:
# Train the two models
#logistic
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
#nn
ffnn = create_ffnn() 
ffnn.fit(X_train, y_train, epochs=epochs, verbose=0)

# Accuracy: rate of correct classifications:
#logistic
correct_classif = (logreg.predict(X_test) == y_test).astype(int)
print("LR acc   :", np.mean(correct_classif))
#nn
y_pred = np.array(ffnn.predict(X_test) > .5, dtype=int)[:, 0]
correct_classif = (y_pred == y_test).astype(int)
print("NN acc   :", np.mean(correct_classif))

# Plot boundaries
plot_decision_surface(model=logreg, x=X_test, y=y_test)
plot_decision_surface(model=ffnn,   x=X_test, y=y_test)

#### What's wrong?

Let's Look at the confusion matrix.

A confusion matrix is a table that summarizes the performance of a classification model by comparing the predicted labels with the actual labels. The confusion matrix shows the number of true positives (TP), true negatives (TN), false positives (FP), and false negatives (FN) for each class.

<div style="text-align:center;">
    <img src="https://media.geeksforgeeks.org/wp-content/uploads/20221031145731/ConfusionMatrix2.png" width="40%">
</div>


1. True Negative – Indicates how many negative values are predicted as negative only by the model
2. False Positive – Indicates how many negative values are predicted as positive values by the model
3. False Negative – Indicates how many positive values are predicted as negative values by the model
4. True Positive – Indicates how many positive values are predicted as positive only by the model

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

# the NN Confusion matrix
print(f'accuracy score: {accuracy_score(y_test, y_pred)}')
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

In [ ]:
# Plot split data
plt.subplot(121)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train);
plt.subplot(122)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test);

We did not shuffled the data.

In [ ]:
# Shuffle the data!
# take some random permutations of N -> randomly permute betweeen 0 and N
p = np.random.permutation(N)
idx_train = p[:n]
idx_test = p[n:]

# Data split
X_train, y_train = X[idx_train], y[idx_train]
X_test, y_test = X[idx_test], y[idx_test]

# Plot split data
plt.subplot(121)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train);
plt.subplot(122)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test);


SkLearn provides many [utilities](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection). For example, `train_test_split`, `ShuffleSplit` and `StratifiedShuffleSplit`.


In [ ]:
# or with SkLearn
from sklearn.model_selection import train_test_split
idx_train, idx_test = train_test_split(np.arange(N), test_size=0.2, shuffle=True)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# Data split
X_train, y_train = X[idx_train], y[idx_train]
X_test, y_test = X[idx_test], y[idx_test]

# Plot split data
plt.subplot(121)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train);
plt.subplot(122)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test);

In [ ]:
# Train the two models
#logistic
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
#nn
ffnn = create_ffnn(8) 
ffnn.fit(X_train, y_train, epochs=epochs, verbose=0)
y_pred = np.array(ffnn.predict(X_test) > .5, dtype=int)[:, 0]

# Evaluate accuracy
acc_lr = logreg.score(X_test, y_test)
[loss_nn, acc_nn] = ffnn.evaluate(X_test, y_test)
print("acc_lr", acc_lr)
print("acc_nn", acc_nn)

# Plot boundaries
plot_decision_surface(model=logreg, x=X_test, y=y_test)
plot_decision_surface(model=ffnn, x=X_test, y=y_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

print(f'accuracy score: {accuracy_score(y_test, y_pred)}')
cf_mat = confusion_matrix(y_test, y_pred)
print('Confusion matrix')
print(cf_mat)
ConfusionMatrixDisplay.from_predictions(y_test, y_pred)

ROC (Receiver Operating Characteristic) curve is a graphical representation of the performance of a binary classification model.

The ROC curve plots the true positive rate (sensitivity) against the false positive rate (1-specificity) at various classification thresholds.

The area under the ROC curve (AUC) is a scalar value that represents the performance of the binary classification model. 
AUC ranges between 0 and 1, where 0 represents a model that makes all predictions wrong, and 1 represents a perfect model that makes all predictions correctly.

Interpretation of the ROC curve and AUC:

- The closer the ROC curve is to the upper-left corner of the plot, the better the performance of the binary classification model.
- If the ROC curve is a diagonal line, it means that the model performs no better than random chance.
- An AUC of 0.5 indicates that the model performs no better than random chance, while an AUC of 1 indicates perfect classification.

AUC can be interpreted as the probability that the model will correctly classify a randomly chosen positive instance higher than a randomly chosen negative instance.

In summary, the ROC curve and AUC are useful tools for evaluating the performance of binary classification models. 

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

def plot_roc_curve(true_y, y_prob, name):
    """
    plots the roc curve based of the probabilities
    """
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr, label=name)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

plot_roc_curve(y_test, ffnn.predict(X_test), 'FFNN')
print(f'FFNN AUC score: {roc_auc_score(y_test, ffnn.predict(X_test) )}')


plot_roc_curve(y_test, logreg.predict(X_test), 'Logistic Regression')
print(f'FFNN AUC score: {roc_auc_score(y_test, logreg.predict_proba(X_test)[::,1] )}')

plt.legend()

## Can we say which model is the best?

What is a T-test?

The null hypothesis of a T-test is a statement that there is no significant difference between the means of two groups.

T-test is done by building the sample mean and the sample variance for the classification case.

There are two types of T-tests: 
- unpaired samples T-test: used when the two groups being compared are independent of each other
- paired samples T-test:  used when the two groups are dependent or related

In our case we will be testing on the same test set, so we investigate the application of the paired T-test


$$
e_i = 
\begin{cases}
1, & \text{if } y_i =   f(x_i;\hat \theta)\\
0, & \text{if } y_i \ne f(x_i;\hat \theta)
\end{cases}
$$

$$\overline e = \frac{1}{l}\sum_{i=1}^l e_i ;\qquad s^2 = \overline e (1 - \overline e)$$

$$T = \frac{\overline e_{nn} - \overline e_{lr}}
           {\sqrt{ \frac{s^2_{nn}}{l} + \frac{s^2_{lr}}{l}}}$$


We want to see if we can fulfill the null hypothesis. We need to check if we are in a 95% confidence interval --> if the results of the statistics is inside the interval (-1.96, 1.96).           

In [ ]:
# Logistic Regression part
e_lr = (y_test == logreg.predict(X_test)).astype(int)
mean_e_lr = e_lr.mean()
s2_lr = mean_e_lr * (1 - mean_e_lr)
print("mean: {} -- s2: {}".format(mean_e_lr, s2_lr))

# Neural Net part
y_pred = (ffnn.predict(X_test) > .5)[:, 0].astype(int)
e_nn = (y_test == y_pred).astype(int)
mean_e_nn = e_nn.mean()
s2_nn = mean_e_nn * (1 - mean_e_nn)
print("mean: {} -- s2: {}".format(mean_e_nn, s2_nn))

# Test statistics
T = (mean_e_nn - mean_e_lr) 
T /= np.sqrt( s2_nn / l + s2_lr / l )
print("is T={} in 95\% confidence interval (-1.96, 1.96) ?".format(T))

# t-test
from scipy.stats import ttest_ind
tt, p_val = ttest_ind(e_lr, e_nn, equal_var=False)
print('t-test: T={:.2f}, p-value={:.4f}'.format(tt, p_val))

# paired t-test
from scipy.stats import ttest_rel
tt, p_val = ttest_rel(e_lr, e_nn)
print('t-test: T={:.2f}, p-value={:.4f}'.format(tt, p_val))

## Did we finish?

- The best model was the neural net,
- We estimated its performance,
- ...

We retrain the best model on the entire dataset.


In [ ]:
ffnn_final = create_ffnn()  # Remember: train the model from scratch.
ffnn_final.fit(X, y, epochs=epochs, verbose=0)

from keras.models import save_model 
save_model(ffnn_final, "my_final_model.tf")

from keras.models import load_model 
loaded_model = load_model("my_final_model.tf")

# Check they are actually the same
print(ffnn_final.evaluate(X, y))
print(loaded_model.evaluate(X, y))

---


## K-fold cross-validation

Say we have a single model and we want to identify a confidence interval for its accuracy.

### Split the data: cross-validation

Cross-validation is a technique used in machine learning to assess the performance of a model. In addition to making sure that we do not have bias in our dataset. It help us to be sure that whatever is been learnt on the training set it is gonna be generalized to the test set.

It involves splitting the available data into training and validation sets, training the model on the training set, and then evaluating its performance on the validation set. This process is repeated several times with different splits of the data to obtain a more robust estimate of the model's performance.


There are several types of cross-validation techniques, including:

- k-fold cross-validation: This involves dividing the data into k equally sized folds, training the model k times, and using a different fold as the validation set each time.
- Stratified cross-validation: This is used when the data is imbalanced, and ensures that the class distribution is preserved in the training and validation sets.

The main idea behind cross-validation is that each observation in our dataset has the opportunity of being tested. In each round, we split the dataset into  k parts: one part is used for validation, and the remaining  k−1 parts are merged into a training subset for model evaluation. The figure below illustrates the process of 5-fold cross-validation:


<div style="text-align:center;">
    <img src="https://www.mltut.com/wp-content/uploads/2020/05/cross-validation.png" width="30%">
</div>

$$ Fold:  k_i $$

$$ Train:  D_{−k_i}$$
 
$$ Evaluate: {x_i, y_i} \in D_{k_i}$$

$$Performance = \frac{1}{N}  \sum_{i=1}^N   {Performance_i} $$ 

 
where:

$D_{-k_i}$ is the training set with the $i$-th fold removed

$D_{k_i}$ is the validation set consisting of the $i$-th fold

$\text{Performance}_i$ is the performance metric (e.g. accuracy) on the $i$-th fold



In [ ]:
from sklearn.model_selection import StratifiedKFold

# Utility to split the data
kfcv = StratifiedKFold(n_splits=10, shuffle=True)
fold_iterator = kfcv.split(X, y)

# Utility to split the data
acc_nn = []

for idx_train, idx_val in fold_iterator:

    # split data
    X_train, y_train = X[idx_train], y[idx_train]
    X_val, y_val = X[idx_val], y[idx_val]

    # train model
    ffnn = create_ffnn(8)  # Remember: train the model from scratch.
    ffnn.fit(X_train, y_train, epochs=epochs, verbose=0)

    # evaluate model
    _, current_acc = ffnn.evaluate(X_val, y_val)
    acc_nn.append(current_acc)

print("Acc list:", acc_nn)
print("This is our estimated accuracy:  {:.3f} +- {:.3f}".format(np.mean(acc_nn), np.std(acc_nn)))

Here, we will have a set of ten models, but the question is which one to select?

We have two additional alternatives:

- Use all the data for training:
    - build a new model M, this model will be a richer model in term of information, it will be better in in term of probability than the other models simply because we are considering more data.

- Ensemble of models:
    - We possess k models and we can employ all of them and establish a voting mechanism (A voting mechanism is a technique used in ensemble modeling where multiple models are combined to make a single prediction. Each model in the ensemble is allowed to make its own prediction based on the input data, and then a final prediction is made by taking a vote among the individual predictions.)
    - Ensemble models are more resilient since they are less affected by individual data points, but the downside is the computational expense.  
<div style="text-align:center;">
    <img src="https://raw.githubusercontent.com/Project-MONAI/tutorials/9b796e43e527f29c6b8563b573513dff0fd86d98/figures/models_ensemble.png" width="50%">
</div>

Even from here we could have compared the two models.

In [ ]:
# Utility to split the data
kfcv = StratifiedKFold(n_splits=10, shuffle=True)
fold_iterator = kfcv.split(X, y)

# Utility to split the data
acc_nn = []
acc_lr = []

for idx_train, idx_val in fold_iterator:

    X_train, y_train = X[idx_train], y[idx_train]
    X_val, y_val = X[idx_val], y[idx_val]

    ffnn = create_ffnn(8)  # Remember: train the model from scratch.
    ffnn.fit(X_train, y_train, epochs=epochs, verbose=0)

    logreg = LogisticRegression() 
    logreg.fit(X_train, y_train)

    _, current_acc = ffnn.evaluate(X_val, y_val)
    acc_nn.append(current_acc)
    current_acc = logreg.score(X_val, y_val)
    acc_lr.append(current_acc)

print("LogReg list:   ", acc_lr)
print("NeuralNet list:", acc_nn)

print("LogReg:     {:.3f} +- {:.3f}".format(np.mean(acc_lr), np.std(acc_lr)))
print("NeuralNet:  {:.3f} +- {:.3f}".format(np.mean(acc_nn), np.std(acc_nn)))

# Paired two sample test
T, p_val = ttest_rel(acc_lr, acc_nn)
print('t-test: T={:.2f}, p-value={:.4f}'.format(T, p_val))
print("is T={:.2f} in 95\% confidence interval (-1.96, 1.96) ?".format(T))

## More than two models and hyper-parameter tuning

In the context of neural networks, a parameter refers to a set of weights and biases that are learned during the training process. 
A hyperparameter, on the other hand, is a configuration variable that is set before the training process begins. These variables affect the behavior of the training algorithm itself, and can have a significant impact on the performance of the resulting model. 

Examples of hyperparameters include:
    - learning rate
    - the number of hidden layers
    - the number of neurons in each layer
    - the activation function used in each layer

The performance of a neural network model heavily depends on the choice of hyperparameters: Hyperparameters can be adjusted in order to find the best combination for a given problem. This process is often referred to as hyperparameter tuning, and can be done using various techniques such as grid search, random search, etc..

It can be a time-consuming process as it requires training multiple models with different hyperparameter configurations. However, it's a crucial step in building a successful machine learning model as it can significantly improve its performance and generalization ability.

<div style="text-align:center;">
    <img src="https://miro.medium.com/v2/resize:fit:640/format:webp/1*5mStLTnIxsANpOHSwAFJhg.png" width="40%">
</div>


We will learn how to apply the grid search hyper-parameter tuning on the number of neurons and the activation function.

In [ ]:
# neurons - activation
model_parameters = [(3, "tanh"), 
                    (3, "relu"), 
                    (6, "tanh"), 
                    (6, "relu")]  

<div style="text-align:center;">
    <img src="https://studymachinelearning.com/wp-content/uploads/2019/10/summary_activation_fn.png" width="50%">
</div>

### Data split

The model is trained on the training set and evaluated on the testing set to assess its performance on unseen data.
The validation set is used to tune hyperparameters such as the learning rate, and number of hidden units, etc.

<div style="text-align:center;">
    <img src="https://b1739487.smushcdn.com/1739487/wp-content/uploads/2021/04/train-and-test-1-min-1.png?lossy=0&strip=1&webp=1" width="35%">
</div>

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)
X_atr, X_val, y_atr, y_val = train_test_split(X_train, y_train, test_size=0.3, shuffle=True)

epochs = 200

# Model selection
acc_list = []
for (neurons, activation) in model_parameters:
    print("Training NN with {} neurons and {} activation".format(neurons, activation))

    model = create_ffnn(neurons=neurons, activation=activation)
    model.fit(X_atr, y_atr, epochs=epochs, verbose=0)

    _, acc = model.evaluate(X_val, y_val)
    acc_list.append(acc)

imax = np.argmax(acc_list)
print("Best model parameters:", model_parameters[imax])

# Performance of best model
(neurons, activation) = model_parameters[imax]
best_model = create_ffnn(neurons=neurons, activation=activation)
best_model.fit(X_train, y_train, epochs=epochs, verbose=0)
_, final_acc = best_model.evaluate(X_test, y_test, verbose=0)
print("Best model accuracy:", final_acc)

# Final trained model
final_model = create_ffnn(neurons=neurons, activation=activation)
final_model.fit(X, y, epochs=epochs, verbose=0);

In [ ]:
model = create_ffnn(neurons=3, activation='tanh')
model.fit(X_train, y_train, epochs=epochs, verbose=0)
_, final_acc = model.evaluate(X_test, y_test, verbose=0)
print("model accuracy:", final_acc)